___

<p style="text-align: center;"><img src="https://docs.google.com/uc?id=1lY0Uj5R04yMY3-ZppPWxqCr5pvBLYPnV" class="img-fluid" alt="CLRSWY"></p>

___



<h1 style="font-size:300%;background-color:red;text-align: center;">Audit Prediction with DL</h1>




In [ ]:
import pyforest

In [ ]:
df=pd.read_csv("Audit data.csv")

* https://data.mendeley.com/datasets/mmcczp3g3y/3

In [ ]:
df.head()

In [ ]:
df.info()

# EDA 

In [ ]:
df.isnull().sum().sort_values().tail(10)
# check num missing values   

In [ ]:
# check number of unique values in features
for col in df.columns:
    print(col,":",df[col].nunique())

In [ ]:
# Detect and define columns created from other columns with the help of suplementary pdf
for col in df.select_dtypes(exclude="object").columns:
    print(col,":",df[col].sum())

In [ ]:
AOP_list = pd.Series([1,5,9,12,15,22,111,113,116,124,201,213,214,219,220,223,224,229,230,336,337,338,339,343,610,615,622,633,638,650,671,680,688])

In [ ]:
drop_AOP = AOP_list.apply(lambda x: "AOP"+str(x) if len(str(x))==3 else 
                          ("AOP00"+str(x) if len(str(x))==1 else "AOP0"+str(x)) )

drop_AOP

In [ ]:
# Drop columns that are created from others
df.drop(drop_AOP,axis=1,inplace=True)

In [ ]:
# Check target column distribution
df.Audit_opinion.value_counts(dropna=False)

In [ ]:
df.Audit_opinion_1_code.value_counts(dropna=False)

In [ ]:
# Focus on missing values
df.Auditor_switch.value_counts(dropna=False)

In [ ]:
df.dropna( how ="any", subset=["Audit_opinion"]).isnull().sum().sort_values()

In [ ]:
# Fill nan values in Auditor_switch with "unknown" to save 4088 rows
df["Auditor_switch"]=df.Auditor_switch.fillna("unknown")

In [ ]:
# Drop rows that contains missing value in target column
df.dropna( how ="any", subset=["Audit_opinion"],inplace=True)

In [ ]:
df.isnull().sum().sort_values()

In [ ]:
# Drop rest of rows which contains missing values
df.dropna(inplace=True)

In [ ]:
df.isnull().sum().sort_values()

In [ ]:
df.shape

In [ ]:
pd.set_option("max_info_columns", 500)
df.info()
pd.reset_option("max_info_columns")

In [ ]:
df

In [ ]:
# Convert "Year_of_establishment" feature object to datetime
df["Year_of_establishment"] = pd.to_datetime(df["Year_of_establishment"])

In [ ]:
df["Year_of_establishment"].dt.year

In [ ]:
# Transform "Year_of_establishment" column into only year 
df["Year_of_establishment"] = df["Year_of_establishment"].dt.year

In [ ]:
# Create a new column "company_age" and fill it with the ages of the companies in the report year
df["company_age"] = df["Year"] - df["Year_of_establishment"]
df["company_age"]

In [ ]:
# Drop year related columns to reduce complexity
df.drop(["Year", "Year_of_establishment"],axis=1, inplace=True)

In [ ]:
# Check and drop columns that you assume prediction power is too low.
pd.set_option("max_info_columns", 500)
df.info()
pd.reset_option("max_info_columns")

In [ ]:
for col in df.columns:
    print(col,":",df[col].nunique())

In [ ]:
df.drop(["Company_id","Unique_identifier","Company_name"],axis=1,inplace=True)

In [ ]:
pd.set_option("max_info_columns", 500)
df.info()
pd.reset_option("max_info_columns")

In [ ]:
df.drop(["CEO_id","Auditor_name","Auditor_id"],axis=1,inplace=True)

In [ ]:
# Detect and correct the type of object columns
df.select_dtypes(include="object")

In [ ]:
df.head()

In [ ]:
df["Industry_code"] = df["Industry_code"].astype("str")
df["Bankruptcy"] = df["Bankruptcy"].astype("str")
df["Big4"] = df["Big4"].astype("str")
df["Restructuring"] = df["Restructuring"].astype("str")

In [ ]:
df.select_dtypes(include="object")

In [ ]:
pd.set_option("max_info_columns", 500)
df.info()
pd.reset_option("max_info_columns")

In [ ]:
df.describe()

In [ ]:
# Check the corelation between target and feature columns
import plotly.express as px
fig = px.bar(x=df.corr()["Audit_opinion_1_code"].sort_values(), y= df.corr()["Audit_opinion_1_code"].sort_values().index)
fig.show()
# df.corr()["Audit_opinion_1_code"].sort_values().plot.barh(figsize=(20,20))

In [ ]:
# Check multicollinearity
fig = px.imshow(df.corr(),
                x= df.select_dtypes(exclude="object").columns,
                y= df.select_dtypes(exclude="object").columns
               )
fig.update_xaxes(side="top")
fig.show()

In [ ]:
# Check multicollinearity
plt.figure(figsize=(20,20))
sns.heatmap(df.corr(),cmap="coolwarm")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from sklearn.metrics import classification_report, confusion_matrix

# Data Preprocessing 

In [ ]:
# No need to use values attribute bc we will scale the data
X = df.drop(["Audit_opinion_1_code","Audit_opinion"],axis=1)
y = df.Audit_opinion_1_code

In [ ]:
X = pd.get_dummies(X,drop_first=True)

In [ ]:
X.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                    test_size=0.20,random_state=6)

In [ ]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model-1 (Straight Forward)

In [ ]:
model = Sequential()
model.add(Dense(48, activation="relu"))
model.add(Dense(24, activation="relu"))
model.add(Dense(12, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["Recall"])

In [ ]:
model.fit(x=X_train_scaled, y=y_train, epochs=100, 
          validation_data=(X_test_scaled, y_test), verbose=1)

## Evaluate-1

In [ ]:
model_loss = pd.DataFrame(model.history.history)
model_loss.head()

In [ ]:
model_loss.plot(figsize=(15,6))

In [ ]:
predictions = model.predict_classes(X_test_scaled)
predictions_train = model.predict_classes(X_train_scaled)

In [ ]:
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

print(confusion_matrix(y_train, predictions_train))
print(classification_report(y_train, predictions_train))

#  Model-2 (Early Stop)

In [ ]:
model = Sequential()
model.add(Dense(48, activation="relu"))
model.add(Dense(24, activation="relu"))
model.add(Dense(12, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["Recall"])

In [ ]:
pd.Series(y_train).value_counts(normalize=True)

In [ ]:
pd.Series(y_train).value_counts(normalize=True)

weights = {0:32, 1:68}
weights

In [ ]:
early_stop = EarlyStopping(monitor="val_loss", mode="auto", verbose=1, 
                           patience = 50,restore_best_weights=True)

In [ ]:
model.fit(x=X_train_scaled, y=y_train, epochs=150, 
          validation_data=(X_test_scaled, y_test), 
          verbose=1,class_weight = weights, callbacks=[early_stop])

## Evaluate-2

In [ ]:
model_loss = pd.DataFrame(model.history.history)
model_loss.tail()

In [ ]:
model_loss.plot(figsize=(15,6))

* https://machinelearningmastery.com/learning-curves-for-diagnosing-machine-learning-model-performance/

In [ ]:
predictions = model.predict_classes(X_test_scaled)
predictions_train = model.predict_classes(X_train_scaled)

print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

print(confusion_matrix(y_train, predictions_train))
print(classification_report(y_train, predictions_train))

# Model-3 (Early Stop with Drop Out)

In [ ]:
model = Sequential()
model.add(Dense(24, activation="relu"))
model.add(Dropout(0.5))

model.add(Dense(24, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(12, activation="relu"))
model.add(Dropout(0.4))


model.add(Dense(1, activation="sigmoid"))

opt1 = tf.keras.optimizers.Adam(learning_rate=0.001)
opt2 = tf.keras.optimizers.Adamax(learning_rate=0.001)
model.compile(loss="binary_crossentropy", optimizer=opt1, metrics=["Recall"])

In [ ]:
early_stop = EarlyStopping(monitor="val_loss", mode="min", 
                           verbose=1, patience = 25, restore_best_weights=True)

In [ ]:
model.fit(x=X_train_scaled, y=y_train, epochs=600, 
          validation_data=(X_test_scaled, y_test), 
          verbose=1, class_weight = weights, callbacks=[early_stop])

## Evaluate-3

In [ ]:
model_loss = pd.DataFrame(model.history.history)
model_loss.tail()

In [ ]:

model_loss.plot(figsize=(15,6))

In [ ]:
predictions = model.predict_classes(X_test_scaled)
predictions_train = model.predict_classes(X_train_scaled)

print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

print(confusion_matrix(y_train, predictions_train))
print(classification_report(y_train, predictions_train))

---
---

# ML Comparision

## XGBoost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb = XGBClassifier()

In [ ]:
xgb.fit(X_train_scaled,y_train)

In [ ]:
predictions = xgb.predict(X_test_scaled)
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

## My lovely Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
log_mod = LogisticRegression(class_weight="balanced", max_iter=5000)

In [ ]:
log_mod.fit(X_train_scaled,y_train)

In [ ]:
predictions = log_mod.predict(X_test_scaled)
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))